In [2]:
import time
import pandas as pd
import os  # CSV 저장을 위한 폴더/파일 관리를 위해 import
# from selenium import webdriver
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

def clean_filename(filename):
    """파일 이름으로 사용할 수 없는 문자를 '_'로 대체합니다."""
    invalid_chars = '\\/*?:"<>|().'
    cleaned_name = filename.strip() # 양쪽 공백 제거
    for char in invalid_chars:
        cleaned_name = cleaned_name.replace(char, '_')
    cleaned_name = cleaned_name.replace(' ', '_') # 공백도 '_'로 대체
    # 혹시 모를 연속된 '_'를 하나로 변경 (선택적)
    while "__" in cleaned_name:
        cleaned_name = cleaned_name.replace("__", "_")
    return cleaned_name

def scrape_full_preview(driver, wait, scroll_wait):
    """
    [최종 수정됨] StaleElementReferenceException을 방지하기 위해
    루프 내에서 tbody를 계속 갱신하고, 파싱 직전에도 갱신합니다.
    """
    print("    [Sub] '데이터 미리보기' 탭의 그리드 로드 대기 중...")
    try:
        # 그리드 헤더와 본문이 로드될 때까지 대기
        wait.until(EC.visibility_of_element_located((By.ID, 'sheet1_Header')))
        # 초기 tbody 확인 (이 변수를 루프에서 재사용하지 않음)
        initial_tbody = wait.until(EC.visibility_of_element_located((By.ID, 'sheet1_Data')))
        print("    [Sub] 초기 데이터 로드 완료.")

        if "데이터 준비중입니다." in initial_tbody.text or "데이터가 없습니다." in initial_tbody.text:
            print("    [Sub] '데이터 준비중'이거나 데이터가 없어 이 항목을 스킵합니다.")
            return None

    except TimeoutException:
        print("    [Sub] 데이터 그리드(sheet1_Data)를 찾는 데 실패했습니다. (미리보기 미제공 추정)")
        return None

    # 1. 헤더(컬럼명) 추출
    header_elements = driver.find_elements(By.XPATH, "//thead[@id='sheet1_Header']//th")
    headers = [h.text for h in header_elements if h.text.strip() not in ['', 'No']]

    # 2. 모든 데이터 로드를 위한 스크롤 반복
    last_row_count = 0
    while True:
        try:
            # [수정] 스크롤 전에 매번 tbody를 새로 찾습니다.
            tbody = driver.find_element(By.ID, 'sheet1_Data')
            current_row_count = len(tbody.find_elements(By.TAG_NAME, 'tr'))

            # [수정] 이전 행 개수와 비교하여 더 이상 로드되지 않으면 탈출
            if current_row_count == last_row_count:
                print(f"    [Sub] 스크롤 완료. 총 {current_row_count}개 행 로드됨.")
                break

            last_row_count = current_row_count
            print(f"      [Sub] 스크롤... 현재 {current_row_count}개 행 로드됨.")

            # JavaScript를 실행하여 tbody의 스크롤을 맨 아래로 내림
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", tbody)

            # 스크롤 후 새 행이 로드될 때까지 (행 개수가 늘어날 때까지) 대기
            scroll_wait.until(
                lambda d: len(d.find_element(By.ID, 'sheet1_Data').find_elements(By.TAG_NAME, 'tr')) > current_row_count
            )

            time.sleep(0.5) # DOM 안정화를 위한 잠깐의 대기

        except TimeoutException:
            # 5초 동안 새 행이 로드되지 않으면(TimeoutException 발생)
            # 모든 데이터를 로드한 것으로 간주하고 루프 종료
            print(f"    [Sub] 스크롤 시간 초과(정상 종료). 총 {last_row_count}개 행 로드됨.")
            break
        except Exception as e:
            print(f"    [Sub] 스크롤 중 예외 발생: {e}")
            break # 스크롤 중 다른 오류 발생 시 중지

    # 3. 스크롤이 완료된 후, 전체 데이터 파싱
    print("    [Sub] 전체 데이터 파싱 시작...")
    all_data = []

    try:
        # [수정] 파싱 직전에도 tbody를 "반드시" 새로 찾습니다.
        final_tbody = driver.find_element(By.ID, 'sheet1_Data')
        data_rows = final_tbody.find_elements(By.TAG_NAME, 'tr')

        for row in data_rows:
            cells = row.find_elements(By.XPATH, './td[position()>1]')
            all_data.append([cell.text for cell in cells])

    except Exception as e:
        print(f"    [Sub] 최종 데이터 파싱 중 오류 발생: {e}")
        return None # 파싱 실패 시 None 반환

    # 4. Pandas DataFrame으로 변환
    df = pd.DataFrame(all_data, columns=headers)
    return df

# --- 메인 크롤링 로직 ---

# 크롤링할 URL
url = "https://www.bigdata-environment.kr/user/data_market/detail.do?id=1711a0a0-2f03-11ea-bccd-b704c648ae09"

# 제외할 파일 제목
EXCLUDED_TITLE = "행정동별 제과/아이스크림분야 소비인구"

# CSV 파일이 저장될 디렉토리 이름
OUTPUT_DIR = "scraped_data"

# Chrome WebDriver 설정 (자동 설치)
# service = Service(ChromeDriverManager().install())
# [수정] Chrome WebDriver 설정 부분
# 안정적인 실행을 위해 Chrome 옵션을 추가합니다.
chrome_options = Options()
chrome_options.add_argument("--no-sandbox") # 샌드박스 모드 비활성화 (권한 충돌 방지)
chrome_options.add_argument("--disable-dev-shm-usage") # 리소스 부족 문제 방지 (주로 Linux/Docker에서 필요하나 Windows에서도 도움됨)
chrome_options.add_argument("--disable-gpu") # GPU 가속 비활성화 (GPU 드라이버 충돌 방지)
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36") # 버전은 적당히 최신으로
# chrome_options.add_argument("--headless") # (선택사항) 아예 브라우저 창을 띄우지 않고 백그라운드에서 실행

print("Chrome 드라이버를 옵션과 함께 실행합니다...")

# [수정] 드라이버 설정 시 options=chrome_options 를 전달합니다.
# (이전 코드: driver = webdriver.Chrome())
# driver = webdriver.Chrome(options=chrome_options)
driver = uc.Chrome(options=chrome_options)

# 대기 시간 설정
wait = WebDriverWait(driver, 10) # 일반 대기 (10초)
scroll_wait = WebDriverWait(driver, 5) # 스크롤 후 새 데이터 로드 대기 (5초)

# 스크랩한 모든 데이터를 저장할 딕셔너리
all_scraped_data = {}

print(f"페이지 로드 중: {url}")
driver.get(url)

try:
    # '제공데이터' 목록이 있는 <ul> 요소를 찾습니다.
    data_list_container = wait.until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, "div.data-provide-list > ul"))
    )

    # <li> 태그(각 파일 항목)의 총 개수를 셉니다.
    list_items = data_list_container.find_elements(By.TAG_NAME, "li")
    item_count = len(list_items)
    print(f"'제공데이터' 섹션에서 총 {item_count}개의 항목을 찾았습니다.")

    # 주의: 0부터 시작 (i = 0은 첫 번째 항목)
    for i in range(item_count):
        print(f"\n--- [Main] 항목 {i+1}/{item_count} 처리 중 ---")

        try:
            data_list_container = driver.find_element(By.CSS_SELECTOR, "div.data-provide-list > ul")
            current_item = data_list_container.find_elements(By.TAG_NAME, "li")[i]

            title_element = current_item.find_element(By.TAG_NAME, "dt")
            title = title_element.text.strip()
            print(f"  [Main] 제목: {title}")
        except Exception as e:
            print(f"  [Main] {i+1}번째 항목을 찾는 데 실패했습니다: {e}. 다음 항목으로 넘어갑니다.")
            continue

        # 1. 제외할 제목인지 확인
        if title == EXCLUDED_TITLE:
            print(f"  [Main] 제목이 '{EXCLUDED_TITLE}'와 일치하므로 스킵합니다.")
            continue

        # 2. 항목 클릭 (제목(dt)을 클릭하여 해당 파일 선택)
        try:
            print(f"  [Main] 항목 클릭: '{title}'")
            title_element.click()

            wait.until(
                EC.element_to_be_clickable((By.ID, "IBS_data_preview_Button"))
            )
            time.sleep(1) # AJAX 로딩을 위한 추가 대기

        except Exception as e:
            print(f"  [Main] 항목 클릭 중 오류 발생: {e}. 이 항목을 스킵합니다.")
            continue

        # 3. '데이터 미리보기' 탭 클릭
        try:
            print("  [Main] '데이터 미리보기' 탭 클릭...")
            preview_tab = driver.find_element(By.ID, "IBS_data_preview_Button")
            preview_tab.click()
        except Exception as e:
            print(f"  [Main] '데이터 미리보기' 탭 클릭 중 오류 발생: {e}. 이 항목을 스킵합니다.")
            continue

        # 4. 스크롤 및 스크래핑 함수 호출
        df = scrape_full_preview(driver, wait, scroll_wait)

        if df is not None:
            print(f"  [Main] '{title}' 항목의 데이터 {len(df)}행 스크랩 완료.")
            all_scraped_data[title] = df
        else:
            print(f"  [Main] '{title}' 항목에서 데이터를 스크랩하지 못했습니다 (미리보기 없음).")

    # --- 모든 루프 종료 ---
    print("\n=========================================")
    print("모든 크롤링 작업 완료.")

    if not all_scraped_data:
        print("스크랩된 데이터가 없습니다. (제외 항목만 있었거나 미리보기가 없는 항목뿐이었습니다.)")
    else:
        print(f"총 {len(all_scraped_data)}개의 파일에서 데이터를 스크랩했습니다.")

        # --- CSV 저장 로직 [추가된 부분] ---

        # 출력 디렉토리 생성 (없으면 만들기)
        os.makedirs(OUTPUT_DIR, exist_ok=True)
        print(f"\n'{OUTPUT_DIR}' 폴더에 CSV 파일로 저장합니다...")

        for title, df in all_scraped_data.items():
            # 파일명으로 부적절한 문자 제거/변경
            safe_title = clean_filename(title)
            file_path = os.path.join(OUTPUT_DIR, f"{safe_title}.csv")

            try:
                # CSV 파일로 저장 (UTF-8 with BOM으로 Excel 호환성 확보)
                df.to_csv(file_path, index=False, encoding='utf-8-sig')
                print(f"  [저장 완료] '{title}' -> {file_path}")
            except Exception as e:
                print(f"  [저장 실패] '{title}' 저장 중 오류 발생: {e}")

        print("\n--- 저장 작업 완료 ---")
        # --- CSV 저장 로직 끝 ---

except Exception as e:
    print(f"전체 프로세스 중 심각한 오류가 발생했습니다: {e}")

finally:
    # 모든 작업이 끝나면 브라우저 종료
    print("브라우저를 닫습니다.")
    driver.quit()

Chrome 드라이버를 옵션과 함께 실행합니다...
페이지 로드 중: https://www.bigdata-environment.kr/user/data_market/detail.do?id=1711a0a0-2f03-11ea-bccd-b704c648ae09
전체 프로세스 중 심각한 오류가 발생했습니다: Message: 
Stacktrace:
	GetHandleVerifier [0x0x34fe43+66515]
	GetHandleVerifier [0x0x34fe84+66580]
	(No symbol) [0x0x13dc48]
	(No symbol) [0x0x188704]
	(No symbol) [0x0x188aab]
	(No symbol) [0x0x1cf482]
	(No symbol) [0x0x1ab214]
	(No symbol) [0x0x1ccba7]
	(No symbol) [0x0x1aafc6]
	(No symbol) [0x0x17c2ca]
	(No symbol) [0x0x17d154]
	GetHandleVerifier [0x0x5a7353+2521315]
	GetHandleVerifier [0x0x5a22d3+2500707]
	GetHandleVerifier [0x0x377c94+229924]
	GetHandleVerifier [0x0x3681f8+165768]
	GetHandleVerifier [0x0x36ecad+193085]
	GetHandleVerifier [0x0x358158+100072]
	GetHandleVerifier [0x0x3582f0+100480]
	GetHandleVerifier [0x0x3425aa+11066]
	BaseThreadInitThunk [0x0x76765d49+25]
	RtlInitializeExceptionChain [0x0x77d1d6db+107]
	RtlGetAppContainerNamedObjectPath [0x0x77d1d661+561]
	(No symbol) [0x0]

브라우저를 닫습니다.
